In [ ]:
# default_exp evaluate

In [ ]:
%load_ext autoreload
%autoreload 2

# Evaluate

> This module contains functions to infer on the test set and generate the final submission file for Kaggle.

In [ ]:
#hide
from nbdev.showdoc import *
from plant_pathology.utils import *
from plant_pathology.dataset import *
from wwf.vision.timm import *
import timm

In [ ]:
#export
from fastai.vision.all import *
from typing import *

## Infer on Test Set

In [ ]:
#export
def infer_on_test_set(learn: Learner, tta: bool=False) -> Tensor:
    path_test = Path("~/kaggle/plant-pathology/data/plant-pathology-2020/test.csv")
    df_test = pd.read_csv(path_test)
    test_dl = learn.dls.test_dl(df_test)
    preds, _ = (learn.tta if tta else learn.get_preds)(dl=test_dl)
    return preds

In [ ]:
#hide
path, df = load_data()
dls = get_dls(path, df, presize=32, resize=32, bs=512)

In [ ]:
learn = cnn_learner(dls, resnet18)
preds = infer_on_test_set(learn, tta=True)

KeyboardInterrupt: 

In [ ]:
test_close(preds.sum(1), 1.)  # cnn_learner applies softmax after tta

KeyboardInterrupt: 

In [ ]:
timm_learn = timm_learner(dls, "seresnext26d_32x4d")
preds = infer_on_test_set(timm_learn, tta=True)

In [ ]:
preds

tensor([[8.7005e-01, 1.8312e-03, 1.2757e-01, 5.4920e-04],
        [6.1381e-01, 2.4086e-01, 1.3975e-01, 5.5811e-03],
        [9.9843e-01, 1.1407e-03, 3.3470e-04, 9.0708e-05],
        ...,
        [7.8401e-01, 3.3753e-02, 1.6062e-01, 2.1611e-02],
        [7.9484e-01, 2.0938e-02, 1.5667e-01, 2.7547e-02],
        [8.1854e-01, 5.1929e-03, 1.7611e-01, 1.5711e-04]])

## Format Submission

In [ ]:
#export
def format_submission(preds: Tensor, save_path: Union[Path, str]) -> Path:
    data_path = Path("/home/jupyter/kaggle/plant-pathology/data/plant-pathology-2020/")
    submission = pd.read_csv(data_path/"sample_submission.csv")
    
    # Update cols with preds
    submission["healthy"] = preds[:, 0]
    submission["multiple_diseases"] = preds[:, 1]
    submission["rust"] = preds[:, 2]
    submission["scab"] = preds[:, 3]
    
    # Make parent dirs
    save_path = Path(save_path)
    Path(save_path.parent).mkdir(parents=True, exist_ok=True)
    submission.to_csv(save_path, index=False)
    return save_path

In [ ]:
fake_preds = torch.zeros(1821, 4)
save_path = format_submission(fake_preds, "TESTING.csv"); save_path

Path('TESTING.csv')

In [ ]:
assert save_path.exists()

In [ ]:
save_path.unlink()

## All-in-One Learner -> Submission Function

In [ ]:
#export
def evaluate(learn: Learner, save_path: Union[Path, str]=Path("./submission.csv"), tta: bool=False) -> Path:
    """Takes trained learner, evaluates on test set, formats and saves submission.csv."""
    preds = infer_on_test_set(learn, tta=tta)
    return format_submission(preds, save_path)

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_dataset.ipynb.
Converted 02_evaluate.ipynb.
Converted 03_train.ipynb.
Converted 04_generate_pseudo_labels.ipynb.
Converted 05_self_knowledge_distillation.ipynb.
Converted index.ipynb.
